In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import quandl
import calendar
import os
os.chdir('H:\W-GITSYNCH')


%matplotlib inline
sns.set(style="whitegrid")

In [2]:
vix = pd.read_csv("^VIX.csv", index_col="Date")
vix.index = pd.to_datetime(vix.index, format="%Y-%m-%d")
vix = vix.apply(pd.to_numeric, errors="coerce")

spx = pd.read_csv("^GSPC.csv", index_col="Date")
spx.index = pd.to_datetime(spx.index, format="%Y-%m-%d")
spx = spx.apply(pd.to_numeric, errors="coerce")

def DatesAndPct(df):
    df["day"] = df.index.dayofyear
    df["month"] = df.index.month
    df["year"] = df.index.year
    df["pct"] = np.log(df["Adj Close"]).diff(1)
    return df

vix = DatesAndPct(vix)
spx = DatesAndPct(spx)

In [25]:
spx_vix = pd.concat([vix['Adj Close'], spx['Adj Close']], axis = 1)
spx_vix.columns = ['vix','spx']

def DatesAndPct1(df,dtevx,dtespx):
    df["day"] = df.index.dayofyear
    df["month"] = df.index.month
    df["year"] = df.index.year
    v = "pct vix " + str(dtevx) + " day"
    s = "pct spx " + str(dtespx) + " day"
    df[v] = np.log(df["vix"]).diff(dtevx)
    df[s] = np.log(df["spx"]).diff(dtespx)
    return df

final_vix_spx = DatesAndPct1(spx_vix,1,4).dropna()
final_vix_spx.to_csv('final_vix_spx.csv')

In [17]:
def worst_return(vixlevel,dte):
    df = final_vix_spx.copy()
    df["pct spx dte"] = np.log(df["spx"]).diff(dte)
    df = df[df['vix'] < vixlevel].dropna()
    df = df.sort_values(['pct spx dte'], ascending = 1)
    return df

In [29]:
#2 days is 3 data points 
worst_return(12.15,1).head(5)

,vix,spx,day,month,year,pct vix 1 day,pct spx 4 day,pct spx dte
Date,,,,,,,,
2007-01-25,11.22,1423.900024,25,1,2007,0.126174,-0.004624,-0.011334
2005-06-23,12.13,1200.729980,174,6,2005,0.093251,-0.013426,-0.010892
1995-06-26,11.80,544.130005,177,6,1995,0.059354,-0.001561,-0.010203
2005-12-27,11.57,1256.540039,361,12,2005,0.119189,-0.002448,-0.009599
2017-06-29,11.44,2419.699951,180,6,2017,0.131535,-0.007658,-0.008637
